# Import `pygismo` and other packages

We import `pygismo`

In [ ]:
import pygismo as gs

We also import `numpy` and `matplotlib`

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

# Constructing a 2D basis

We use the same B-spline basis as in the previous example, with knot vectors:

$\Xi_u = \Xi_v = \{0,0,0,0.25,0.50,0.75,1,1,1\}$


In [ ]:
kv_u = kv_v = gs.nurbs.gsKnotVector(np.array([0,0,0,0.25,0.50,0.75,1,1,1]),2)
tbasis = gs.nurbs.gsTensorBSplineBasis2(kv_u,kv_v)

# Loading the point data

The data for fitting is stored in two files: `shiphull_pars.csv` and `shiphull_points.csv`. We read them into `numpy.matrix` objects as follows:


In [ ]:
pars   = np.loadtxt("filedata/shiphull_pars.csv",delimiter=",")
points = np.loadtxt("filedata/shiphull_points.csv",delimiter=",")

# Fitting a surface through the point cloud

As for the curve fitting problem, we use `pygismo.modelling.gsFitting` to solve the fitting problem:

In [ ]:
fitter = gs.modelling.gsFitting(pars,points,tbasis)

Then, the fit is computed by calling `compute()`

In [ ]:
fitter.compute()

And the result is obtained by calling `result()`.

In [ ]:
fit = fitter.result()

print(fit.coefs())

In [ ]:
predictions = fit.eval(pars)
mse = ((predictions - points) ** 2).mean()
print(mse)

# Plotting the result in Python

Now we plot the result using `matplotlib`. The plotting procedure is as in the other examples, i.e. we use a `numpy.meshgrid` to define our evaluation points:

In [ ]:
N = M = 100
x = np.linspace(0,1,N)
y = np.linspace(0,1,M)

XX, YY = np.meshgrid(x,y,indexing='xy')
pts = np.stack((XX.flatten(),YY.flatten()))

S = fit.eval(pts)
XX = S[0,:].reshape((N,M))
YY = S[1,:].reshape((N,M))
ZZ = S[2,:].reshape((N,M))

fig = plt.figure()
ax = fig.add_subplot(projection ='3d')
ax.plot_surface(XX,YY,ZZ,cmap=cm.coolwarm)
ax.scatter(points[0,:],points[1,:],points[2,:],color="red")

ax.axis('equal')
ax.axis('off')

fig.tight_layout()
plt.show()

# Plotting the result in Paraview

Alternatively, we can use Paraview to plot our surface. We simply use `pygismo`'s buit-in functionality

In [ ]:
gs.io.gsWriteParaview(fit,"output/fit")
pointsT = points.T
gs.io.gsWriteParaviewPoints(np.matrix(pointsT[:,0]),np.matrix(pointsT[0,:]),np.matrix(pointsT[0,:]),"output/points")

This creates the files `surf0.vts` and `surf.pvd`. When we open `surf.pvd` in paraview, we see the surface.